<a href="https://colab.research.google.com/github/ariel07-sw/applied-data-in-econ/blob/main/5220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

In [2]:
FILE_PATH = "Asset_Class_Returns.xlsx"
SHEET_NAME = "rtns"
DATE_COL = None

In [3]:
assets = ["SPY", "IWM (R2k)", "VWO (EM)", "AGG", "JNK", "GLD"]
mu = np.array([0.075, 0.080, 0.085, 0.020, 0.040, 0.030])

In [4]:
RISK_CAP = 0.12
ANNUAL_FACTOR = 12

In [18]:
df = pd.read_excel("asset class returns.xlsx", sheet_name="rtns")

assets = ["SPY", "IWM (R2k)", "VWO (EM)", "AGG", "JNK", "GLD"]

rets = df[assets].copy()
rets = rets.apply(pd.to_numeric, errors="coerce")
rets = rets.dropna()

In [19]:
cov_monthly = rets.cov()
cov_annual = cov_monthly * 12

In [21]:
rets.shape

(175, 6)

In [22]:
import cvxpy as cp
import numpy as np

In [23]:
Sigma = cov_annual.values
n = len(mu)

In [33]:
def optimize_portfolio(lb, ub):
    w = cp.Variable(n)

    portfolio_var = cp.quad_form(w, Sigma)
    portfolio_vol = cp.sqrt(portfolio_var)

    objective = cp.Maximize(mu @ w)

    constraints = [
        cp.sum(w) == 1,
        w >= lb,
        w <= ub,
        portfolio_var <= 0.12**2
    ]

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return w.value

In [34]:
w_lo = optimize_portfolio(0.0, 0.30)
w_ls = optimize_portfolio(-0.30, 0.30)

In [35]:
ret_lo = mu @ w_lo
vol_lo = np.sqrt(w_lo.T @ Sigma @ w_lo)

In [36]:
def port_stats(w):
    port_ret = float(mu @ w)
    port_vol = float(np.sqrt(w.T @ Sigma @ w))
    return port_ret, port_vol

In [37]:
ret_lo, vol_lo = port_stats(w_lo)
ret_ls, vol_ls = port_stats(w_ls)

In [38]:
def show(title, w, ret, vol):
    out = pd.DataFrame({"Asset": assets, "Weight": w})
    out["Weight_%"] = out["Weight"] * 100

    print("\n" + "="*70)
    print(title)
    print(f"Expected Return (annual): {ret*100:.2f}%")
    print(f"Expected Risk   (annual): {vol*100:.2f}%")
    print("-"*70)
    print(out[["Asset", "Weight_%"]].to_string(index=False, formatters={"Weight_%": "{:.2f}".format}))
    print("="*70)

show("A) Long-Only (0% to 30%), Risk <= 12%", w_lo, ret_lo, vol_lo)
show("B) Long-Short (-30% to 30%), Risk <= 12%", w_ls, ret_ls, vol_ls)


A) Long-Only (0% to 30%), Risk <= 12%
Expected Return (annual): 5.98%
Expected Risk   (annual): 12.00%
----------------------------------------------------------------------
    Asset Weight_%
      SPY    30.00
IWM (R2k)    11.14
 VWO (EM)    24.99
      AGG    30.00
      JNK     0.00
      GLD     3.87

B) Long-Short (-30% to 30%), Risk <= 12%
Expected Return (annual): 6.10%
Expected Risk   (annual): 12.00%
----------------------------------------------------------------------
    Asset Weight_%
      SPY    30.00
IWM (R2k)    20.68
 VWO (EM)    22.21
      AGG    30.00
      JNK   -20.43
      GLD    17.53


In [39]:
import pandas as pd

In [40]:
df = pd.DataFrame({
    "Country": ["U.K.", "Japan", "U.S.", "Germany"],
    "w_bench": [0.15, 0.30, 0.45, 0.10],   # MSCI benchmark weights
    "w_mgr":   [0.30, 0.10, 0.40, 0.20],   # manager weights
    "R_mgr":   [0.20, 0.15, 0.10, 0.05],   # manager return in country
    "R_idx":   [0.12, 0.15, 0.14, 0.12],   # country index return
})

In [41]:
df

,Country,w_bench,w_mgr,R_mgr,R_idx
0,U.K.,0.15,0.3,0.20,0.12
1,Japan,0.30,0.1,0.15,0.15
2,U.S.,0.45,0.4,0.10,0.14
3,Germany,0.10,0.2,0.05,0.12


In [43]:
R_mgr_total = (df["w_mgr"] * df["R_mgr"]).sum()
R_bench_total = (df["w_bench"] * df["R_idx"]).sum()
active_return = R_mgr_total - R_bench_total

In [44]:
df["allocation"] = (df["w_mgr"] - df["w_bench"]) * df["R_idx"]
allocation_total = df["allocation"].sum()

In [45]:
df["selection"] = df["w_mgr"] * (df["R_mgr"] - df["R_idx"])
selection_total = df["selection"].sum()


In [46]:
check = allocation_total + selection_total

In [47]:
print("Manager Return:", round(R_mgr_total*100, 2), "%")
print("Benchmark Return:", round(R_bench_total*100, 2), "%")
print("Active Return (Mgr - Bench):", round(active_return*100, 2), "%\n")

print("Allocation Effect:", round(allocation_total*100, 2), "%")
print("Selection Effect:", round(selection_total*100, 2), "%")
print("Allocation + Selection:", round(check*100, 2), "%\n")

print(df[["Country","allocation","selection"]])

Manager Return: 12.5 %
Benchmark Return: 13.8 %
Active Return (Mgr - Bench): -1.3 %

Allocation Effect: -0.7 %
Selection Effect: -0.6 %
Allocation + Selection: -1.3 %

   Country  allocation  selection
0     U.K.       0.018      0.024
1    Japan      -0.030      0.000
2     U.S.      -0.007     -0.016
3  Germany       0.012     -0.014
